### This is Part 1 of LinkedIn Posts scrapping. In this notebook Selenium + Python has been used. This code extracts the posts from LinkedIn based on given keywords. While running this notebook, you need to provide LinkedIn link generated by searching a word on LinkedIn after running first cell. After running second cell, you need to provide your LinkedIn Id & pass. Please also note that this code is for Windows system.

In [ ]:
#required installs (i.e. pip3 install in terminal): pandas, selenium, bs4

from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re as re
import random
#import caffeine
#caffeine.on(display=True)

page = input("Enter the Company Linkedin URL: ")
company_name = page[33:-1]

try:
    f= open("credentials.txt","r")
    contents = f.read()
    username = contents.replace("=",",").split(",")[1]
    password = contents.replace("=",",").split(",")[3]
except:
    f= open("credentials.txt","w+")
    username = input('Enter your linkedin username: ')
    password = input('Enter your linkedin password: ')
    f.write("username={}, password={}".format(username,password))
    f.close()

# Output after running this code block - "Enter the Company Linkedin URL:" - Paste this URL (https://www.linkedin.com/search/results/content/?keywords=stalk&origin=SWITCH_SEARCH_VERTICAL)
#                                                                            in the box and press Enter
# You will also get same URL if you login to your LinkedIn and give 'stalk' in search bar of LinkedIn and select Posts.

In [ ]:
#access Webriver
#browser = webdriver.Chrome('chromedriver')
browser = webdriver.Chrome(r'C:\Users\sanni\Downloads\applications\chromedriver_win32\chromedriver')

#Open login page
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Enter login info:
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)
#Note: replace the keys "username" and "password" with your LinkedIn login info. Please provide the Username & Password in quotes('')
elementID.submit()

In [ ]:
# #Go to webpage
browser.get(page)

In [ ]:
def scroll():
    #Simulate scrolling to capture all posts
    SCROLL_PAUSE_TIME = random.randint(1,3)

    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")
    #print(last_height)

    while True:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [ ]:
def scroll_up():
    #Simulate scrolling to capture all posts
    SCROLL_PAUSE_TIME = random.randint(1,3)

    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(document.body.scrollHeight, 0);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [ ]:
def scrape_post(container):
    
        global post_dates 
        global post_texts 
        global post_likes
        global post_comments
        global video_views 
        global media_links 
        global media_type 
    
        try:
            try:
                post_date = container.find("span",{"class":"feed-shared-actor__sub-description t-12 t-normal t-black--light"}) 
                post_dates.append(post_date.text.strip()[0:-2])
            except:
                post_dates.append("None")
                
            try:
                text_box = container.find("div",{"class":"feed-shared-update-v2__description-wrapper ember-view"})   
                post_texts.append(text_box.text.strip())
            
            except:
                post_texts.append("None")
            
            try:
                new_likes = container.findAll("li", {"class":"social-details-social-counts__reactions social-details-social-counts__item"})
                new_comments = container.findAll("li", {"class": "social-details-social-counts__comments social-details-social-counts__item"})
            except:
                new_likes = 'none'
                new_comments = "none"

            try:
                video_box = container.findAll("div",{"class": "feed-shared-update-v2__content feed-shared-linkedin-video ember-view"})
                video_link = video_box[0].find("video", {"class":"vjs-tech"})
                media_links.append(video_link['src'])
                media_type.append("Video")
            except:
                try:
                    image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                    image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image feed-shared-image__image--constrained lazy-image ember-view"})
                    media_links.append(image_link['src'])
                    media_type.append("Image")
                except:
                    try:
                        #mutiple shared images
                        image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                        image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image lazy-image ember-view"})
                        media_links.append(image_link['src'])
                        media_type.append("Multiple Images")
                    except:
                        try:
                            article_box = container.findAll("div",{"class": "feed-shared-article__description-container"})
                            article_link = article_box[0].find('a', href=True)
                            media_links.append(article_link['href'])
                            media_type.append("Article")
                        except:
                            try:
                                video_box = container.findAll("div",{"class": "feed-shared-external-video__meta"})          
                                video_link = video_box[0].find('a', href=True)
                                media_links.append(video_link['href'])
                                media_type.append("Youtube Video")   
                            except:
                                try:
                                    poll_box = container.findAll("div",{"class": "feed-shared-update-v2__content overflow-hidden feed-shared-poll ember-view"})
                                    media_links.append("None")
                                    media_type.append("Other: Poll, Shared Post, etc")
                                except:
                                    media_links.append("None")
                                    media_type.append("Unknown")



            #Getting Video Views. (The folling three lines prevents class name overlap)
            view_container2 = set(container.findAll("li", {'class':["social-details-social-counts__item"]}))
            view_container1 = set(container.findAll("li", {'class':["social-details-social-counts__reactions","social-details-social-counts__comments social-details-social-counts__item"]}))
            result = view_container2 - view_container1

            view_container = []
            for i in result:
                view_container += i

            try:
                video_views.append(view_container[1].text.strip().replace(' Views',''))

            except:
                video_views.append('N/A')


            try:
                post_likes.append(new_likes[0].text.strip())
            except:
                post_likes.append(0)
                pass

            try:
                post_comments.append(new_comments[0].text.strip())                           
            except:                                                           
                post_comments.append(0)
                pass

        except:
            pass

    

In [ ]:
# post_dates = []
# post_texts = []
# post_likes = []
# post_comments = []
# video_views = []
# media_links = []
# media_type = []

# browser.switch_to.window(browser.window_handles[1])
# post_source = get_source()
# containers = post_source.findAll("div",{"class":"feed-shared-update-v2 feed-shared-update-v2--minimal-padding full-height relative artdeco-card ember-view"})
# container = containers[0]



# try:
#     try:
#         post_date = container.find("span",{"class":"feed-shared-actor__sub-description t-12 t-normal t-black--light"}) 
#         post_dates.append(post_date.text.strip()[0:-2])
#     except:
#         post_dates.append("None")

#     try:
#         text_box = container.find("span",{"class":"entity-result__summary entity-result__no-ellipsis mt3 t-14 t-black"})
#         if text_box == None:
#             text_box = container.find("p",{"class":"entity-result__content-summary entity-result__no-ellipsis t-14"}) 

#         post_texts.append(text_box.text.strip())

#     except:
#         post_texts.append("None")

#     try:
#         new_likes = container.findAll("li", {"class":"social-details-social-counts__reactions social-details-social-counts__item"})
#         new_comments = container.findAll("li", {"class": "social-details-social-counts__comments social-details-social-counts__item"})
#     except:
#         new_likes = 'none'
#         new_comments = "none"

#     try:
#         video_box = container.findAll("div",{"class": "feed-shared-update-v2__content feed-shared-linkedin-video ember-view"})
#         video_link = video_box[0].find("video", {"class":"vjs-tech"})
#         media_links.append(video_link['src'])
#         media_type.append("Video")
#     except:
#         try:
#             image_box = container.findAll("div",{"class": "feed-shared-image__container"})
#             image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image feed-shared-image__image--constrained lazy-image ember-view"})
#             media_links.append(image_link['src'])
#             media_type.append("Image")
#         except:
#             try:
#                 #mutiple shared images
#                 image_box = container.findAll("div",{"class": "feed-shared-image__container"})
#                 image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image lazy-image ember-view"})
#                 media_links.append(image_link['src'])
#                 media_type.append("Multiple Images")
#             except:
#                 try:
#                     article_box = container.findAll("div",{"class": "feed-shared-article__description-container"})
#                     article_link = article_box[0].find('a', href=True)
#                     media_links.append(article_link['href'])
#                     media_type.append("Article")
#                 except:
#                     try:
#                         video_box = container.findAll("div",{"class": "feed-shared-external-video__meta"})          
#                         video_link = video_box[0].find('a', href=True)
#                         media_links.append(video_link['href'])
#                         media_type.append("Youtube Video")   
#                     except:
#                         try:
#                             poll_box = container.findAll("div",{"class": "feed-shared-update-v2__content overflow-hidden feed-shared-poll ember-view"})
#                             media_links.append("None")
#                             media_type.append("Other: Poll, Shared Post, etc")
#                         except:
#                             media_links.append("None")
#                             media_type.append("Unknown")



#     #Getting Video Views. (The folling three lines prevents class name overlap)
#     view_container2 = set(container.findAll("li", {'class':["social-details-social-counts__item"]}))
#     view_container1 = set(container.findAll("li", {'class':["social-details-social-counts__reactions","social-details-social-counts__comments social-details-social-counts__item"]}))
#     result = view_container2 - view_container1

#     view_container = []
#     for i in result:
#         view_container += i

#     try:
#         video_views.append(view_container[1].text.strip().replace(' Views',''))

#     except:
#         video_views.append('N/A')


#     try:
#         post_likes.append(new_likes[0].text.strip())
#     except:
#         post_likes.append(0)
#         pass

#     try:
#         post_comments.append(new_comments[0].text.strip())                           
#     except:                                                           
#         post_comments.append(0)
#         pass

# except:
#     pass


In [ ]:
def get_source():  
    company_page = browser.page_source
    linkedin_soup = bs(company_page.encode("utf-8"), "html")
    #linkedin_soup.prettify()

    return linkedin_soup


In [ ]:

def get_posts(links):
    
    for link in links:

        link.click()
        time.sleep(3)

        #switch to post page and get source
        browser.switch_to.window(browser.window_handles[1])
        post_source = get_source()
        
        
        post_block = post_source.findAll("div",{"class":"core-rail update-outlet"})
        time.sleep(2)
        
        #scrape the post and then close and switch back to main page
        scrape_post(post_block[0])
        browser.close()
        browser.switch_to.window(browser.window_handles[0])
        time.sleep(random.randint(4,24))

        

In [ ]:
def get_post_links():
    #scroll to end of list
    scroll()
    links = []
    hrefs = []
    
    #find all post links
    elems = browser.find_elements_by_xpath("//a[@href]")
    for elem in elems:
        #check if the link is a feed
        if 'https://www.linkedin.com/feed/update/' in elem.get_attribute('href'): 
            links.append(elem)
            hrefs.append(elem.get_attribute('href'))
        
        
    scroll_up()
    return links, hrefs

In [ ]:
def get_unique_links(links, hrefs):
    mask = [True]
    
    for idx, link in enumerate(hrefs[1:], 1):
        if link == hrefs[idx-1]:
            mask.append(False)
        else:
            mask.append(True)
            
    unique_links = [elem for bol,elem in zip(mask, links) if bol]
    
    return unique_links

In [ ]:
post_dates = []
post_texts = []
post_likes = []
post_comments = []
video_views = []
media_links = []
media_type = []

def scrape_pages():
    page = 1
    page_limit = 10
    
    while page < page_limit:

        links, hrefs = get_post_links()
        #print('links:\n',links)
        
        unique_links = get_unique_links(links, hrefs)

        get_posts(unique_links)
        #print('post_dates:\n',post_dates)
        
        try:
            browser.find_element_by_xpath('//*[@class="artdeco-pagination__button artdeco-pagination__button--next artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--1 artdeco-button--tertiary ember-view"]').click()
            time.sleep(random.randint(1,5))
            page += 1
        except:
            print("All pages scraped")
            break   
        

In [ ]:
scrape_pages()

In [ ]:
cleaned_dates = []

for i in post_dates:
    d = str(i[0:3]).replace('\n\n', '').replace('•','').replace(' ', '')
    cleaned_dates += [d]
    
# comment_count = []
# for i in post_comments:
#     s = str(i).replace('Comment','').replace('s','').replace(' ','')
#     comment_count += [s]

In [ ]:
#pd.set_option('max_colwidth', 1000)

data = {
    "Date Posted": cleaned_dates,
    "Media Type": media_type,
    "Post Text": post_texts,
    "Post Likes": post_likes,
    "Post Comments": post_comments,
    "Video Views": video_views,
    "Media Links": media_links
}


df = pd.DataFrame(data)
df

In [ ]:
media_type

In [ ]:
#write original dataframe to excel file
#df.to_csv("{}_posts.csv".format(company_name), encoding='utf-8', index=False)

writer = pd.ExcelWriter("{}_posts.xlsx".format('#affordablehousing'), engine='xlsxwriter')
df.to_excel(writer, index =False)
writer.save()

In [ ]:
#write unique dataframe to csv file
unique_df.to_csv('gross_linkedin_scrape.csv')